In [ ]:
import tensorflow as tf
from tensorflow_tts.inference import AutoProcessor
from IPython.display import Audio

In [ ]:
processor = AutoProcessor.from_pretrained(pretrained_path="./kss_mapper.json")
tacotron2_tflite_model = 'models/tacotron2_korean-dynmRange.tflite'
mb_melgan_model = 'mb_melgan_korean_saved_model'

In [ ]:
mb_melgan = tf.saved_model.load(mb_melgan_model)

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tacotron2_tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_text = "신은 우리의 수학 문제에는 관심이 없다. 신은 다만 경험적으로 통합할 뿐이다."
input_ids = processor.text_to_sequence(input_text)

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], [1, len(input_ids)])
interpreter.allocate_tensors()
input_data = (tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
          tf.convert_to_tensor([len(input_ids)], tf.int32),
          tf.convert_to_tensor([0], dtype=tf.int32))

for i, detail in enumerate(input_details):
    input_shape = detail['shape']
    interpreter.set_tensor(detail['index'], input_data[i])
    
interpreter.invoke()
decoder_output, mel_output= (interpreter.get_tensor(output_details[0]['index']),interpreter.get_tensor(output_details[1]['index']))

In [ ]:
audios = mb_melgan.inference(mel_outputs)
Audio(data=audios[0, :, 0], rate=22050)